In [1]:
import numpy as np
import os
from keras.models import Model
from keras import layers

import tensorflow as tf

/home/alberto/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def _parse_function(proto):

    keys_to_features = {'sound': tf.FixedLenFeature([], tf.string),
                        "label": tf.FixedLenFeature([], tf.float32)}
    
    # Load one example
    parsed_features = tf.parse_single_example(proto, keys_to_features)
    
    # Turn the sound string back into an array
    parsed_features['sound'] = tf.decode_raw(parsed_features['sound'], tf.float32)
    
    
    return parsed_features['sound'], parsed_features["label"]

In [3]:
def create_dataset(filepath, classification=True):
    
    files = [filepath + "/" + file for file in os.listdir(filepath)]

    dataset = tf.data.TFRecordDataset(files)
    
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_function, num_parallel_calls=4)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
    
    # Set the number of datapoints you want to load and shuffle 
    dataset = dataset.shuffle(500)
    
    # Set the batchsize
    dataset = dataset.batch(batchsize)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    sound, label = iterator.get_next()

    # Bring your picture back in shape
    sound = tf.reshape(sound, [-1, 24000, 1])
    label = tf.reshape(label, [-1, 1])
    
#     Create a one hot array for your labels
#     label = tf.one_hot(label, NUM_CLASSES)

    
    return sound, label

In [4]:
filepath = "./data/tf_data/train"
batchsize = 16
epochs = 5
train_samples = sum([int(file.split("_")[0]) for file in os.listdir(filepath)])
train_samples

26007

In [5]:
sound, label = create_dataset(filepath)

# Model

In [7]:
# inputs = layers.Input(shape=(data.shape[1],1))
inputs = layers.Input(tensor=sound)

conv1 = layers.Conv1D(filters=64, kernel_size=129, padding="same")(inputs)
pool1 = layers.MaxPool1D(8)(conv1)

reshape = layers.Reshape((3000,64,1))(pool1)

conv2 = layers.Conv2D(filters=96, kernel_size=(61,1), padding="same", activation='relu')(reshape)
pool2 = layers.MaxPool2D((8,1))(conv2)
conv3 = layers.Conv2D(filters=128, kernel_size=(35,1), padding="same", activation='relu')(pool2)
pool3 = layers.MaxPool2D((4,1))(conv3)
conv4 = layers.Conv2D(filters=160, kernel_size=(17,1), padding="same", activation='relu')(pool3)
pool4 = layers.MaxPool2D((12,1))(conv4)
avgpool = layers.AveragePooling2D((5,1))(pool4)

flatten = layers.Flatten()(avgpool)

dense1 = layers.MaxoutDense(output_dim=128,nb_feature=2)(flatten)
dropout1 = layers.Dropout(0.5)(dense1)
dense2 = layers.MaxoutDense(output_dim=64, nb_feature=2)(dropout1)
dropout2 = layers.Dropout(0.5)(dense2)

output = layers.Dense(1, activation="sigmoid")(dropout2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/alberto/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:74: UserWarning: The `MaxoutDense` layer is deprecated and will be removed after 06/2017.
  warnings.warn('The `MaxoutDense` layer is deprecated '


In [8]:
model = Model(inputs=inputs, outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24000, 1)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24000, 64)         8320      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3000, 64)          0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 3000, 64, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3000, 64, 96)      5952      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 375, 64, 96)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 375, 64, 128)      430208    
__________

In [8]:
model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              target_tensors=[label])

In [9]:
history = model.fit(epochs=epochs,
                    steps_per_epoch=int(train_samples/batchsize))

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1625/1625 [==============================] - -5754s -3541070us/step - loss: 0.6446 - acc: 0.6376
Epoch 2/5
1625/1625 [==============================] - 1129s 695ms/step - loss: 0.6126 - acc: 0.6719
Epoch 3/5
 869/1625 [===============>..............] - ETA: 8:44 - loss: 0.6250 - acc: 0.6461

KeyboardInterrupt: 

In [105]:
# x_train = data.reshape((data.shape[0], data.shape[1], 1))

In [106]:
# x_train.shape

In [1]:
# history = model.fit(x_train, 
#                     labels,
#                     batch_size=32,
#                     epochs=5,
#                     validation_split=0.1)

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [17]:
import torch

In [18]:
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0
print(cuda.get_device_name(cuda.current_device()))

GeForce GTX 1050 Ti


In [3]:
tf.test.is_gpu_available()

True